# 모델개선_워드임베딩

In [1]:
# 구글 드라이브와 연결

#from google.colab import auth
#auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# 경로 설정
chat_dir = '/content/gdrive/My Drive/Colab Notebooks/충북대 AI특강/실습 파일/[3일차]2.챗봇 실습/practice_data/data/'
!ls -al '/content/gdrive/My Drive/Colab Notebooks/충북대 AI특강/실습 파일/[3일차]2.챗봇 실습/practice_data/data/'
!ls -al '/content/gdrive/My Drive/Colab Notebooks/충북대 AI특강/'

total 983
-rw------- 1 root root 351034 Jul 28 02:01 data_morphed.txt
-rw------- 1 root root 299248 May  1 12:46 data.txt
-rw------- 1 root root  70242 Jul 28 02:01 test_data_morphed.txt
-rw------- 1 root root 280792 Jul 28 02:01 train_data_morphed.txt
drwx------ 2 root root   4096 Jul 19 13:20 word2vec
total 8
drwx------ 2 root root 4096 Jul 28 01:15 '강의 교재'
drwx------ 3 root root 4096 Jul 28 01:13 '실습 파일'


In [3]:
print(chat_dir)

/content/gdrive/My Drive/Colab Notebooks/충북대 AI특강/실습 파일/[3일차]2.챗봇 실습/practice_data/data/


# Word2Vec의 생성과 유의어 찾기

In [4]:
# 형태소분석기 관련 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk

!pip install JPype1
!pip install rhinoMorph

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [5]:
# 학습할 텍스트 읽기 
import os.path

embedding_dim = 50                                       # 임베딩 차원수 설정

os.chdir(chat_dir+'word2vec/')                           # 경로 설정
print("Current Directory:", os.getcwd())

with open('wiki_test.txt', 'r', encoding='utf-8') as f:  # 테스트용 파일 읽기 
    data = f.read()

Current Directory: /content/gdrive/My Drive/Colab Notebooks/충북대 AI특강/실습 파일/[3일차]2.챗봇 실습/practice_data/data/word2vec


In [6]:
# 문장단위 분리 및 형태소 분석기 기동
import rhinoMorph
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

sent_data = sent_tokenize(data)                       # 문장 단위 분리
rn = rhinoMorph.startRhino()                          # RHINO 기동

print('type:', type(sent_data))
print('length:', len(sent_data))           # 전체 문장의 개수
print('sentence sample:', sent_data[:20])  # 형태소 분석 전 모습

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!
type: <class 'list'>
length: 11977
sentence sample: ['<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">\n지미 카터\n\n제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.', '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.', '조지아 공과대학교를 졸업하였다.', '그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다.', '1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다.', '그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.', '1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다.', '대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다.', '조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.', '1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.', '카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.', '카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트

In [7]:
# 텍스트의 형태소 분석
total_lines = len(sent_data)
cnt = 0
with open(chat_dir+'word2vec/wiki202003_nationalcorpus_naverratings_morphed.txt', 'w', encoding='utf-8') as f:
    for data_each in sent_data:
        morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each, pos=['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG'])
        joined_data_each = ' '.join(morphed_data_each)
        if joined_data_each:
            f.write(joined_data_each + '\n')
        cnt += 1
        if (cnt % 1000) == 0:  # 진행정도 확인을 위해 1000번째 문장마다 확인
            print(round(cnt/total_lines * 100, 3), '%')
    print('Morphological Analysis Completed.')

8.349 %
16.699 %
25.048 %
33.397 %
41.747 %
50.096 %
58.445 %
66.795 %
75.144 %
83.493 %
91.843 %
Morphological Analysis Completed.


In [8]:
# 형태소 분석 결과를 읽어 리스트로 만들기
def read_data(filename, encoding='utf-8'):                # 읽기 함수 정의
    with open(filename, 'r', encoding=encoding) as f:
        data = [line.split(' ') for line in f.read().splitlines()]
    return data


data = read_data(chat_dir+"word2vec/wiki202003_nationalcorpus_naverratings_morphed.txt", 'utf-8')

print(len(data))
print(type(data))
print(data[:3])

11976
<class 'list'>
[['URL', '미', '카터', '미', '카터', '제임스', '얼', '미', '카터', '주니어', '민주당', '출신', '미국', '대통령'], ['미', '카터', '조지아', '섬터', '카운티', '마을', '태어나'], ['조지아', '공과대학', '교', '졸업']]


In [9]:
# 임베딩 구성 
from gensim.models import Word2Vec
os.chdir(chat_dir+'word2vec/')

model = Word2Vec(sentences=data, size=embedding_dim, window=10, min_count=5, workers=4, sg=1)
model.save('embedding_window10_mincnt5_skipgram.model')

print('Completed.')

Completed.


In [10]:
# 임베딩 값 저장 
words = list(model.wv.vocab)
with open('embedding_window10_mincnt5_skipgram.txt', 'w') as f:
    for word in words:
        data = model.wv[word].tolist()      # 현재 단어의 임베딩 값을 가져온다
        print('data_pre:', data)            # 현재 단어의 임베딩 값을 출력해본다

        data.insert(0, word)                # 시작 부분에 해당 단어를 넣는다
        print('data_after:', data)          # 현재 단어의 이름과 함께 임베딩 값을 출력해본다 

        for item in data:                   # 단어 이름부터 시작하여 각 벡터의 값을 저장한다
            f.write("%s " % item)
        f.write("\n")

Output hidden; open in https://colab.research.google.com to view.

In [11]:
# 유사어 찾기
#model = Word2Vec.load('embedding_window10_mincnt5_skipgram_big.model')    # test 파일이 아닌, 대량의 데이터로 미리 만들어진 워드 임베딩을 사용한다 
model = Word2Vec.load('embedding_window10_mincnt5_skipgram.model')    # embedding_window10_mincnt5_skipgram.model
print('--- 유사단어 출력 ---')
similarWords = model.wv.most_similar(positive=['행복', '웃음', '밝', '기쁨'], topn=5)
print(similarWords)

word = []
for similarWord in similarWords:
    word.append(similarWord[0])

print(word)

--- 유사단어 출력 ---
[('그대', 0.9764381647109985), ('깜짝', 0.9757905006408691), ('숙연', 0.9726068377494812), ('제발', 0.9723167419433594), ('고뇌', 0.9716861248016357)]
['그대', '깜짝', '숙연', '제발', '고뇌']


In [12]:
# 두 단어 사이의 유사도 계산
print('--- 두 단어의 유사도 계산 ---')
print('한국과 일본:', model.wv.similarity('한국', '일본'))
print('한국과 미국:', model.wv.similarity('한국', '미국'))
print('한국과 중국:', model.wv.similarity('한국', '중국'))

--- 두 단어의 유사도 계산 ---
한국과 일본: 0.6482641
한국과 미국: 0.74009657
한국과 중국: 0.79025835


# 챗봇 모델 개선

In [13]:
# 데이터 읽기 함수 정의
def read_data(filename, encoding='cp949'):
  with open(filename, 'r', encoding=encoding) as f:
    data = [line.split('\t') for line in f.read().splitlines()]
  return data

In [14]:
# 형태소분석기 준비
import rhinoMorph
rn = rhinoMorph.startRhino()         # 형태소분석기 기동

data = read_data(chat_dir+'data.txt', encoding='cp949')

print('자료 타입:', type(data))
print('전체 문장수:', len(data))
print('형태소 분석 전 모습:', data[:20])

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
자료 타입: <class 'list'>
전체 문장수: 6322
형태소 분석 전 모습: [['저희 안경원 코드 번호 확인이 가능할까요?', '매장코드'], ['저희 안경원 코드를 잊어버렸습니다', '매장코드'], ['안경원 코드번호 확인이 어렵습니다 어떻게 해야하나요?', '매장코드'], ['매장 코드번호 확인이 어렵습니다 어떻게 해야하나요?', '매장코드'], ['안경원 코드번호 확인하는 방법이 있을까요?', '매장코드'], ['안경원 코드번호 조회부탁드립니다', '매장코드'], ['저희 안경원 코드 확인부탁드려요', '매장코드'], ['안경원 코드 번호를 잊어버렸어요 확인부탁드려요', '매장코드'], ['안경원 코드 번호 확인을 어디에서 확인 할 수 있나요?', '매장코드'], ['저희 안경원 코드 어떻게 확인 할 수 있나요?', '매장코드'], ['저희 매장 코드 번호 확인하고싶어요', '매장코드'], ['매장 코드 번호 어디에서 볼 수 있나요?', '매장코드'], ['매장 코드 번호 확인하고싶어요 어디에서 볼 수 있나요?', '매장코드'], ['매장 코드 번호 조회는 어디에서 할 수 있나요?', '매장코드'], ['매장 코드 번호 어떻게 알 수 있나요?', '매장코드'], ['매장 번호 확인하고싶어요 어디에서 볼 수 있나요?', '매장코드'], ['코드번호 까먹었습니다', '매장코드'], ['코드번호 잊어버렸습니다', '매장코드'], ['코드번호 잊어버렸을땐 어떻게 해야하죠?', '매장코드'], ['매장 코드 번호 잊어버렸습니다 어떻게 확인하죠?', '매장코드']]


In [15]:
# data 내용 형태소 분석
with open(chat_dir+'data_morphed.txt', 'w', encoding='utf-8') as f:
    for data_each in data:
        morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each[0], pos=['NNG', 'NNP', 'NP', 'VV', 'VA', 'VX', 'XR', 'IC', 'MM', 'MAG'])
        print("morphed_data_each:", morphed_data_each)
        joined_data_each = ' '.join(morphed_data_each)
        if joined_data_each:
            f.write(joined_data_each + '\t' + data_each[1] + '\n')
    print('Morphological Analysis Completed.')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
morphed_data_each: ['어떤', '제품', '교환', '확인', '부탁', '드리']
morphed_data_each: ['교환', '신청', '내역', '조회', '부탁', '드리']
morphed_data_each: ['교환', '신청', '내역', '조회', '주']
morphed_data_each: ['교환', '신청', '내역', '조회', '어디', '하']
morphed_data_each: ['교환', '내역', '조회', '수', '있']
morphed_data_each: ['교환', '내역', '조회', '있']
morphed_data_each: ['교환', '내역', '조회', '하', '수', '있']
morphed_data_each: ['교환', '내역', '조회', '수', '있']
morphed_data_each: ['교환', '내역', '조회', '수', '있']
morphed_data_each: ['교환', '내역', '조회', '있']
morphed_data_each: ['교환', '내역', '조회', '있']
morphed_data_each: ['교환', '내역', '조회', '어떻하', '하']
morphed_data_each: ['교환', '내역', '조회', '어떻하', '하']
morphed_data_each: ['교환', '내역', '조회', '어떻하', '하']
morphed_data_each: ['교환', '내역', '조회', '어떻하', '하']
morphed_data_each: ['교환', '내역', '조회', '어떻하', '하', '수', '있']
morphed_data_each: ['교환', '내역', '조회', '어떻하', '하', '수', '있']
morphed_data_each: ['교환', '내역', '조회', '어떻하', '하', '수', '있']
morphed_data_each: ['교환', '내역', '조회', '어떻하

In [16]:
print("연결된 마지막 문장: ", joined_data_each)
print("마지막 문장의 라벨: ", data_each[1])

연결된 마지막 문장:  제품 배송비 없 주문 금액 얼마
마지막 문장의 라벨:  배송비


In [17]:
def write_data_list(list, filename, encoding):
    """리스트 변수를 위한 쓰기 함수"""
    with open(chat_dir+filename, 'w') as f:
        for item in list:
            f.write('%s\t%s\n' % (item[0], item[1]))

In [18]:
# 훈련데이터와 테스트데이터 분리
from sklearn.model_selection import train_test_split

data = read_data(chat_dir+'data_morphed.txt', encoding='utf-8')
train, test = train_test_split(data, test_size=0.2)

write_data_list(list=train, filename='train_data_morphed.txt', encoding='utf-8')
write_data_list(list=test, filename='test_data_morphed.txt', encoding='utf-8')

In [ ]:
# 훈련 데이터 읽기
data = read_data(chat_dir+'train_data_morphed.txt', encoding='utf-8')
print('train length:', len(data))

texts = [line[0] for line in data]                      # 훈련데이터 본문
labels = [line[1] for line in data]                     # 훈련데이터 레이블 부분

train length: 5057


In [ ]:
# 테스트 데이터 읽기
data_val = read_data(chat_dir+'test_data_morphed.txt', encoding='utf-8')
print('test length:', len(data_val))

texts_val = [line[0] for line in data_val]          # 테스트 데이터 본문
labels_val = [line[1] for line in data_val]         # 테스트 데이터 label 부분

test length: 1265


In [ ]:
# Data Tokenizing
%tensorflow_version 2.x
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import math

max_words = 10000                                   # 데이터셋에서 가장 빈도 높은 10,000 개의 단어만 사용한다
maxlen = 200                                        # 200개 이후의 단어는 버린다
class_number = 10                                   # 분류할 클래스의 수

tokenizer = Tokenizer(num_words=max_words)  	      # 상위빈도 10,000 개의 단어만을 추려내는 Tokenizer 객체 생성
tokenizer.fit_on_texts(texts)                       # 단어 인덱스를 구축한다
word_index = tokenizer.word_index               	  # 단어 인덱스만 가져온다

In [ ]:
# 전체 데이터가 가지고 있는 토큰의 수를 보여준다. texts_to_sequences()를 거쳐야 10,000개만 남는다. 
print('전체에서 %s개의 고유한 토큰을 찾았습니다.' % len(word_index))
print('word_index type: ', type(word_index))
print('word_index: ', word_index)

전체에서 218개의 고유한 토큰을 찾았습니다.
word_index type:  <class 'dict'>
word_index:  {'하': 1, '확인': 2, '어떻하': 3, '제품': 4, '있': 5, '반품': 6, '교환': 7, '어디': 8, '가능': 9, '주': 10, '조회': 11, '달': 12, '수': 13, '이번': 14, '배송': 15, '주문': 16, '내역': 17, '영업': 18, '되': 19, '알리': 20, '샘플': 21, '신청': 22, '언제': 23, '번호': 24, '개봉': 25, '싶': 26, '마감일': 27, '회수': 28, '도착': 29, '당일': 30, '상품': 31, '알': 32, '담당': 33, '금액': 34, '코드': 35, '시간': 36, '받': 37, '드리': 38, '부탁': 39, '배송비': 40, '연락처': 41, '대상': 42, '궁금': 43, '사원': 44, '지': 45, '저희': 46, '일정': 47, '정하': 48, '얼마': 49, '매장': 50, '방법': 51, '보': 52, '안경원': 53, '없이': 54, '한가': 55, '방문': 56, '품목': 57, '여부': 58, '수량': 59, '개수': 60, '담당자': 61, '갯수': 62, '요청': 63, '환불': 64, '능': 65, '자': 66, '소비자': 67, '청구': 68, '이상': 69, '공급가': 70, '수령': 71, '문의': 72, '날짜': 73, '한': 74, '얼마나': 75, '서': 76, '통화': 77, '전화': 78, '청구서': 79, '대금': 80, '법': 81, '동일': 82, '않': 83, '없': 84, '이력': 85, '는지': 86, '저번': 87, '같': 88, '지궁': 89, '확': 90, '서해': 91, '안': 92, '일자': 93, '필요': 94, '붙': 95

In [ ]:
# Data Sequencing
# train 데이터를 Sequencing 한다
# 상위 빈도 10,000개의 단어를 word_index의 숫자 리스트로 변환. Tokenizer 결과가 여기서 반영된다
print(type(texts))
data = tokenizer.texts_to_sequences(texts) 
print('data_pre:', data)

# maxlen의 수만큼으로 2D 텐서를 만든다
data = pad_sequences(data, maxlen=maxlen)

print('data:', data)
print(data.shape)
print(type(data))

<class 'list'>
data_pre: [[14, 12, 21, 111], [7, 17, 2, 9], [6, 22, 17, 2, 8, 1], [15, 23, 19], [25, 1, 4, 6, 1, 26], [46, 18, 44, 41, 32, 26], [18, 33, 66, 188, 189, 20, 10], [6, 28, 23, 19, 3, 32, 13, 5], [14, 12, 21, 60, 8, 2, 1], [6, 17, 11, 3, 2, 1, 5], [46, 50, 35, 24, 112, 119, 20, 10], [3, 7, 17, 11, 1, 13, 5, 20, 10], [6, 9, 4, 3, 2, 13, 89], [64, 34, 2, 1, 10], [6, 42, 57, 48, 45, 5], [7, 28, 73, 3, 2, 13, 5], [6, 28, 47, 2, 106, 139, 39, 38], [15, 23, 19, 32, 13, 5], [6, 34, 8, 2], [18, 44, 56, 63, 3, 1], [46, 109, 35, 24, 112], [64, 34, 49, 2, 8, 1], [25, 1, 4, 6, 9, 58, 8, 2], [7, 17, 11, 3, 1, 13, 5], [14, 12, 7, 27, 3, 2], [46, 33, 18, 44, 41, 32, 26], [30, 15, 4, 29, 36, 3, 2, 5], [46, 53, 37, 13, 5, 21, 59, 2, 10], [31, 16, 23, 37, 52, 5], [18, 61, 41, 2, 3, 1, 19], [6, 42, 4, 3, 2, 1], [7, 28, 73, 3, 2, 13, 5], [16, 1, 4, 15, 11, 3, 1], [15, 11, 1, 26, 8, 2, 13, 5], [25, 1, 4, 7, 3, 1, 13, 5, 20, 10], [4, 15, 11, 1, 13, 5, 51, 5], [25, 31, 7, 19], [21, 59, 2, 10], [16

In [ ]:
# test 데이터를 Sequencing 한다
print(type(texts_val))
data_val = tokenizer.texts_to_sequences(texts_val) 
print('data_pre:', data_val)

# maxlen의 수만큼으로 2D 텐서를 만든다
data_val = pad_sequences(data_val, maxlen=maxlen)

print('data_val:', data_val)
print(data_val.shape)
print(type(data_val))

<class 'list'>
data_pre: [[4, 40, 54, 16, 26, 75, 16, 1, 19], [4, 16, 1, 15, 11, 3, 1], [206, 67, 2, 26, 8, 2, 13, 5], [6, 34, 49, 43], [14, 12, 6, 27, 8, 2, 9, 1], [6, 22, 17, 52, 81, 5], [4, 15, 11, 1, 51, 43], [4, 15, 11, 8, 1, 5], [6, 34, 2, 39, 38], [30, 15, 4, 29, 36, 32, 26], [7, 28, 93, 3, 2, 5], [50, 35, 24, 2, 144, 3, 1, 1], [18, 33, 24, 2, 3, 1], [7, 27, 3, 2], [14, 12, 21, 59, 3, 19], [14, 12, 7, 27, 3, 2], [8, 7, 17, 11, 1, 13, 5], [14, 12, 68, 17, 8, 2, 5], [35, 24, 72, 8, 1, 5, 20, 10], [4, 16, 15, 11, 3, 1, 32, 10], [6, 9, 31, 3, 2, 1, 5], [88, 99, 31], [18, 33, 56, 63, 3, 1, 1], [18, 33, 66, 24, 2, 3, 1, 1], [14, 12, 21, 60, 8, 2, 1], [30, 15, 23, 29, 1], [4, 70, 67, 2, 3, 1, 5], [6, 17, 11, 3, 2, 1], [14, 12, 6, 27, 23], [6, 28, 23, 1], [16, 23, 37, 13, 5, 3, 2, 1], [6, 42, 4, 48, 45, 5, 20, 10], [25, 1, 4, 6, 9, 58, 43], [30, 15, 29, 36, 3, 2, 1], [21, 22, 8, 115, 1, 19, 20, 10], [6, 27, 8, 2], [6, 22, 1, 23, 28, 9, 1, 32, 13, 5], [31, 16, 1, 15, 11, 3, 1], [7, 65, 4

In [ ]:
# 문자열 label을 숫자형 변수로 치환하는 함수
def labelToIint(labels):
    for count, label in enumerate(labels):
        if label == "배송비":
            labels[count] = 0
        elif label == "담당자문의":
            labels[count] = 1
        elif label == "제품가격":
            labels[count] = 2
        elif label == "배송문의":
            labels[count] = 3
        elif label == "매장코드":
            labels[count] = 4
        elif label == "샘플문의":
            labels[count] = 5
        elif label == "제품불일치":
            labels[count] = 6
        elif label == "반품문의":
            labels[count] = 7
        elif label == "교환문의":
            labels[count] = 8
        elif label == "청구금액":
            labels[count] = 9
    return labels

In [ ]:
# 훈련데이터와 테스트데이터의 label을 숫자로 치환
print("훈련데이터 label 치환 전:\n", labels)
labels = labelToIint(labels)
print("치환 후:", labels)

print("\n테스트데이터 label 치환 전:\n", labels_val)
labels_val = labelToIint(labels_val)
print("치환 후:", labels_val)

훈련데이터 label 치환 전:
 ['샘플문의', '교환문의', '반품문의', '배송문의', '반품문의', '담당자문의', '담당자문의', '반품문의', '샘플문의', '반품문의', '매장코드', '교환문의', '반품문의', '반품문의', '반품문의', '교환문의', '반품문의', '배송문의', '반품문의', '담당자문의', '매장코드', '반품문의', '반품문의', '교환문의', '교환문의', '담당자문의', '배송문의', '샘플문의', '배송문의', '담당자문의', '반품문의', '교환문의', '배송문의', '배송문의', '교환문의', '배송문의', '교환문의', '샘플문의', '배송문의', '담당자문의', '담당자문의', '샘플문의', '교환문의', '교환문의', '샘플문의', '배송문의', '담당자문의', '담당자문의', '교환문의', '반품문의', '제품가격', '배송문의', '반품문의', '배송문의', '반품문의', '반품문의', '배송문의', '담당자문의', '샘플문의', '샘플문의', '반품문의', '매장코드', '배송문의', '교환문의', '샘플문의', '배송문의', '매장코드', '담당자문의', '반품문의', '배송문의', '교환문의', '반품문의', '배송문의', '배송문의', '배송문의', '반품문의', '샘플문의', '담당자문의', '교환문의', '청구금액', '교환문의', '반품문의', '담당자문의', '샘플문의', '배송문의', '샘플문의', '반품문의', '배송문의', '배송문의', '배송문의', '교환문의', '교환문의', '담당자문의', '배송문의', '샘플문의', '반품문의', '담당자문의', '교환문의', '교환문의', '교환문의', '청구금액', '반품문의', '반품문의', '교환문의', '교환문의', '교환문의', '담당자문의', '교환문의', '샘플문의', '반품문의', '교환문의', '반품문의', '반품문의', '담당자문의', '반품문의', '반품문의', '배송문의', '교환문의', '배송문의', '매장코드', '반품

In [ ]:
# One-Hot Encoding
# 원-핫 인코딩 함수
def to_one_hot(labels, dimension):
      results = np.zeros((len(labels), dimension))
      for i, label in enumerate(labels):
         results[i, label] = 1.
      return results


# 입력층에는 원-핫 인코딩을 수행하지 않는다
# data = to_one_hot(data, dimension=max_words)
# data_val = to_one_hot(data_val, dimension=max_words)

# 출력층만 원-핫 인코딩을 수행한다 
labels = to_one_hot(labels, dimension=class_number)
labels_val = to_one_hot(labels_val, dimension=class_number)

In [ ]:
# 출력층의 형태
print(labels)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [ ]:
# Train 데이터와 Test 데이터 준비
print('데이터 텐서의 크기:', data.shape)          
print('레이블 텐서의 크기:', labels.shape)        

x_train = data                         
y_train = labels                    
x_val = data_val
y_val = labels_val

데이터 텐서의 크기: (5057, 200)
레이블 텐서의 크기: (5057, 10)


In [ ]:
# 모델 정의하기 - Word2Vec Embedding (1), 임베딩 딕셔너리 로딩
embeddings_index = {}
f = open(os.path.join(chat_dir+'word2vec', 'embedding_window10_mincnt5_skipgram.txt'), encoding='utf-8')   #_big : CP949
for line in f:
    values = line.split()
    word = values[0]                                  # 각 행의 단어
    coefs = np.asarray(values[1:], dtype='float32')   # 각 단어의 임베딩값
    embeddings_index[word] = coefs
f.close()

print('%s개의 단어 벡터를 찾았습니다.' % len(embeddings_index))

9857개의 단어 벡터를 찾았습니다.


In [ ]:
# 모델 정의하기 - Word2Vec Embedding (2), 임베딩 행렬 구성
embedding_matrix = np.zeros((max_words, embedding_dim))     # 0으로 채워진 빈 행렬 구성
for word, i in word_index.items():
    if i < max_words:                                       # max_words 이하의 범위에서 순회
        embedding_vector = embeddings_index.get(word)       # 해당 단어의 임베딩 벡터를 가져온다 
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector          # 임베딩 값을 해당 행렬 위치에 주입한다

In [ ]:
# 모델 정의하기 - Word2Vec Embedding (3), 임베딩 층 쌓기
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()                 		# 모델을 새로 정의

# 임베딩 층. 나중에 임베딩된 입력을 Flatten 층에서 펼치기 위해 input_length(=maxlen)를 지정. 출력층은 (input_dim, input_length, output_dim)가 됨
model.add(layers.Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen))
model.add(layers.Flatten())                     # 3D 임베딩 텐서를 (input_dim, input_length * output_dim) 크기의 2D 텐서로 펼친다

# 은닉층
model.add(layers.Dense(units=32, activation='relu'))

# 출력층. 출력 노드는 분류 개수인 10. 활성화 함수는 sigmoid와 성격이 같으나 다중분류에 사용되는 함수인 softmax 함수를 사용한다. 확률값을 출력
model.add(layers.Dense(units=class_number, activation='softmax'))

In [ ]:
# 모델 정의하기 - Word2Vec Embedding (4), 임베딩 행렬의 값 주입
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [ ]:
# 모델 정의하기, Keras Embedding
# 보다 쉽게 사용할 수 있는 Keras Embedding으로 진행
from tensorflow.keras import models
from tensorflow.keras import layers

embedding_dim = 50                              # 임베딩의 차원을 설정한다. 보통 200까지에서 적절히 설정한다. 
model = models.Sequential()                 		# 모델을 새로 정의

# 임베딩 층. 나중에 임베딩된 입력을 Flatten 층에서 펼치기 위해 input_length(=maxlen)를 지정. 출력층은 (input_dim, input_length, output_dim)가 됨
model.add(layers.Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen))
model.add(layers.Flatten())                     # 3D 임베딩 텐서를 (input_dim, input_length * output_dim) 크기의 2D 텐서로 펼친다

# 은닉층
model.add(layers.Dense(units=32, activation='relu'))

# 출력층. 출력 노드는 분류 개수인 10. 활성화 함수는 sigmoid와 성격이 같으나 다중분류에 사용되는 함수인 softmax 함수를 사용한다. 확률값을 출력
model.add(layers.Dense(units=class_number, activation='softmax'))

In [ ]:
# 모델 요약 출력
model.summary()

In [ ]:
# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

In [ ]:
# Train Model
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))
history_dict = history.history

In [ ]:
# 경로 변경
%cd /content/gdrive/My Drive/pytest/data/

In [ ]:
# multidimensional numpy arrays를 저장할 수 있는 h5 file(HDF) 포맷으로 저장한다
model.save('text_embed_we_model.h5') 	     # 모델 저장

# 훈련데이터에서 사용된 상위빈도 10,000개의 단어로 된 Tokenizer 저장
# 새로 입력되는 문장에서도 같은 단어가 추출되게 한다
import pickle
with open('text_embed_we_tokenizer.pickle', 'wb') as handle:
     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Accuracy and Loss 확인
# history 딕셔너리 안에 있는 정확도와 손실값을 가져와 본다
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

print('Accuracy of each epoch:', acc)
epochs = range(1, len(acc) +1)

In [ ]:
# Plotting Accuracy
# 정확도와 손실값의 변화를 보고, epoch를 어디에서 조절해야 할 지를 가늠한다.
# 정확도가 떨어지는 구간, 손실값이 높게 나타나는 구간을 확인한다
# 데이터가 큰 경우 대개 epoch를 늘려야 최적값에 도달한다
import matplotlib.pyplot as plt

# 정확도 그리기
plt.plot(epochs, acc, 'bo', label='Training Acc')
plt.plot(epochs, val_acc, 'b', label='Validation Acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()            			

In [ ]:
# Plotting Loss
plt.figure()            		# 새로운 그림을 그린다

# 손실값 그리기
plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend()            		

plt.show()

In [ ]:
# Load Model
import os
from tensorflow.keras.models import load_model

filepath = '/content/gdrive/My Drive/pytest/data/'
os.chdir(filepath)
print("Current Directory:", os.getcwd())

loaded_model = load_model('text_embed_we_model.h5')
print("model loaded:", loaded_model)

with open('text_embed_we_tokenizer.pickle', 'rb') as handle:
       loaded_tokenizer = pickle.load(handle)

In [ ]:
# Test Data Evaluation
test_eval = loaded_model.evaluate(x_val, y_val)
print('prediction model loss & acc:', test_eval)

In [ ]:
predictions = loaded_model.predict(x_val)
print('predictions shape:', predictions.shape)
print('prediction 0 shape:', predictions[0].shape)
print('prediction 0 sum', np.sum(predictions[0]))             # 모든 분류에 대한 확률의 총합은 1이 되어야 한다

print('prediction 0 value:', predictions[0])                  # 10개 분류 각각에 대한 확률값
print('prediction 0 max value:', np.argmax(predictions[0]))  	# prediction 0이 가지고 있는 가장 큰 값의 판정 결과
print('prediction 1 max value:', np.argmax(predictions[1]))   # prediction 1이 가지고 있는 가장 큰 값의 판정 결과
print('prediction 2 max value:', np.argmax(predictions[2]))   # prediction 2가 가지고 있는 가장 큰 값의 판정 결과

In [ ]:
# 라벨을 답변으로 치환하는 함수
def intToLabel(label_int):
    labels = ''
    if label_int == 0:
        labels = '20,000원 이상 주문하시면 배송비가 없습니다'
    elif label_int == 1:
        labels = '담당자는 홈페이지에 직원의 이름 또는 사번을 넣으시면 자세한 정보를 알 수 있습니다'
    elif label_int == 2:
        labels = '공급가와 소비자가는 홈페이지 > 직원 ID 로그인 > 물품명 > 가격조회 에서 확인 가능합니다'
    elif label_int == 3:
        labels = '배송에는 보통 2일이 소요되며, 빠른 배송을 선택하시면 1일 안에 책임배달합니다'
    elif label_int == 4:
        labels = '매장 코드는 홈페이지 > 매장정보 에서 확인 가능합니다'
    elif label_int == 5:
        labels = '샘플신청은 홈페이지 > 직원 ID 로그인 > 물품명 > 샘플신청 에서 가능합니다'
    elif label_int == 6:
        labels = '다른 제품이 배송되어 죄송합니다. 홈페이지 > 물품명 > 환불신청 또는 전화 상담 부탁드립니다'
    elif label_int == 7:
        labels = '배송받으신 모든 제품은 7일 안에는 반품이 가능합니다. 홈페이지 > 물품명 > 환불신청 에서 반품 가능합니다'
    elif label_int == 8:
        labels = '배송받으신 모든 제품은 7일 안에는 교환이 가능합니다. 홈페이지 > 물품명 > 교환신청 에서 교환 가능합니다'
    elif label_int == 9:
        labels = '청구금액은 매달 12일 이후에 홈페이지 > 직원 ID 로그인 > 청구금액에서 확인 가능합니다'
    return labels

In [ ]:
# 1문장 예측
user_input = input("내용을 입력하세요: ")
morphed_input, poses = rhinoMorph.wholeResult_list(rn, user_input, pos=['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'VCN', 'MAG'])
text = [morphed_input]

data = loaded_tokenizer.texts_to_sequences(text)
x_test = pad_sequences(data, maxlen=maxlen)

predictions = loaded_model.predict(x_test)
label_int = np.argmax(predictions)
label = intToLabel(label_int)
print(label)

In [ ]:
predictions